#Introduction + Problem Definition
In this notebook, 2 different Convolutional Neural Networks (CNNs) will be implemented and compared on a given datase. This dataset is the mnist dataset, as it is a standard dataset for training and testing a new model or comparing certain models.

The 2 models that I will be comparing are the AlexNet and the ResNet. The reason why I am comparing these models will be explained before they get implemented.

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/DW_data/"

Mounted at /content/gdrive


#Data Preparation
In this section of the notebook, the labels will be converted into an array of integers so that it will be 10-ary classification of the data.

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#Create a binary classification matrix for the neural network.
import keras
import numpy as np

num_classes = 10
def categorize(arr):
  res = []
  for i in range(len(arr)):
    res.append([0 if arr[i] != j else 1 for j in range(num_classes)])
    
  return np.array(res)

y_train = categorize(y_train)
y_test = categorize(y_test)

11490434/11490434 [==============================] - 2s 0us/step


#Build the Models.
In the following 2 code blocks, the AlexNet and the ResNet Models will be built.

The AlexNet model has been choosen because of the benefits that come from the number of layers (depth) of the network itself as it will be able to extract a lot of features from the source images (inputs). Although this comes with the burden of using a lot of computational resources, the use of Google Colab's GPU will reduce this restraint by some margin.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.regularizers import l2

#Initialize the AlexNet model.
alexnet = Sequential()

# Layer 1
alexnet.add(Conv2D(96, (11, 11), input_shape=(28, 28, 1), padding='same', kernel_regularizer=l2(0.0)))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 2
alexnet.add(Conv2D(256, (5, 5), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 3
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(512, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 4
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))

# Layer 5
alexnet.add(ZeroPadding2D((1, 1)))
alexnet.add(Conv2D(1024, (3, 3), padding='same'))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(MaxPooling2D(pool_size=(2, 2)))

# Layer 6
alexnet.add(Flatten())
alexnet.add(Dense(3072))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 7
alexnet.add(Dense(4096))
alexnet.add(BatchNormalization())
alexnet.add(Activation('relu'))
alexnet.add(Dropout(0.5))

# Layer 8
alexnet.add(Dense(num_classes))
alexnet.add(BatchNormalization())
alexnet.add(Activation('softmax'))

ResNet's (Residual Network's) main benefit is that it can drop redundant layers that it does not need. This would allow for the model to be "simplified" in terms of it's layers and would allow for the error to be more easily reduced.

In [ ]:
import tensorflow as tf
def identity_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def convolutional_block(x, filter):
    # copy tensor to variable called x_skip
    x_skip = x
    # Layer 1
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same', strides = (2,2))(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = tf.keras.layers.Activation('relu')(x)
    # Layer 2
    x = tf.keras.layers.Conv2D(filter, (3,3), padding = 'same')(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    # Processing Residue with conv(1,1)
    x_skip = tf.keras.layers.Conv2D(filter, (1,1), strides = (2,2))(x_skip)
    # Add Residue
    x = tf.keras.layers.Add()([x, x_skip])     
    x = tf.keras.layers.Activation('relu')(x)
    return x

def ResNet34(shape = (28, 28, 1), classes = 10):
    # Step 1 (Setup Input Layer)
    x_input = tf.keras.layers.Input(shape)
    x = tf.keras.layers.ZeroPadding2D((3, 3))(x_input)
    # Step 2 (Initial Conv layer along with maxPool)
    x = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    # Define size of sub-blocks and initial filter size
    block_layers = [3, 4, 6, 3]
    filter_size = 64
    # Step 3 Add the Resnet Blocks
    for i in range(4):
        if i == 0:
            # For sub-block 1 Residual/Convolutional block not needed
            for j in range(block_layers[i]):
                x = identity_block(x, filter_size)
        else:
            # One Residual/Convolutional Block followed by Identity blocks
            # The filter size will go on increasing by a factor of 2
            filter_size = filter_size*2
            x = convolutional_block(x, filter_size)
            for j in range(block_layers[i] - 1):
                x = identity_block(x, filter_size)
    # Step 4 End Dense Network
    x = tf.keras.layers.AveragePooling2D((2,2), padding = 'same')(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation = 'relu')(x)
    x = tf.keras.layers.Dense(classes, activation = 'softmax')(x)
    model = tf.keras.models.Model(inputs = x_input, outputs = x, name = "ResNet34")
    return model

#Run and Compare Models

In [ ]:
alexnet.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(learning_rate=0.01),
              metrics=['accuracy'])

alexnet.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

score_alexnet = alexnet.evaluate(x_test, y_test, verbose=0)
print('AlexNet Test loss:', score_alexnet[0])
print('AlexNet Test accuracy:', score_alexnet[1])

In [ ]:
resnet = ResNet34()
resnet.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(learning_rate=0.01),
              metrics=['accuracy'])

resnet.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

score_resnet = resnet.evaluate(x_test, y_test, verbose=0)
print('ResNet Test loss:', score_resnet[0])
print('ResNet Test accuracy:', score_resnet[1])

Epoch 1/5
469/469 [==============================] - 39s 76ms/step - loss: 0.3412 - accuracy: 0.8999 - val_loss: 0.1244 - val_accuracy: 0.9613
Epoch 2/5
469/469 [==============================] - 33s 71ms/step - loss: 0.0800 - accuracy: 0.9760 - val_loss: 0.0892 - val_accuracy: 0.9710
Epoch 3/5
469/469 [==============================] - 34s 72ms/step - loss: 0.0337 - accuracy: 0.9906 - val_loss: 0.0785 - val_accuracy: 0.9735
Epoch 4/5
469/469 [==============================] - 34s 72ms/step - loss: 0.0129 - accuracy: 0.9977 - val_loss: 0.0734 - val_accuracy: 0.9754
Epoch 5/5
469/469 [==============================] - 34s 72ms/step - loss: 0.0056 - accuracy: 0.9995 - val_loss: 0.0713 - val_accuracy: 0.9781
Test loss: 0.07131729274988174
Test accuracy: 0.9781000018119812
